In [3]:
import numpy as np
import pandas as pd
import csv

from scipy.spatial.distance import cdist

from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.algorithms.moo.nsga2 import NSGA2


from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.algorithms.moo.nsga2 import binary_tournament
from pymoo.algorithms.moo.nsga3 import comp_by_cv_then_random

from pymoo.core.problem import ElementwiseProblem

from pymoo.operators.sampling.rnd import PermutationRandomSampling
from pymoo.operators.crossover.ox import OrderCrossover
from pymoo.operators.mutation.inversion import InversionMutation


from pymoo.optimize import minimize
from pymoo.util.ref_dirs import get_reference_directions

In [4]:
kroA100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroA100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroB100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroB100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")
kroC100=pd.read_csv('C:/Users/Maria/Documents/Python Scripts/kroC100.csv', header=0, encoding='utf-8', skip_blank_lines=False, delimiter=r",")

In [8]:
class TravelingSalesman(ElementwiseProblem):
    def __init__(self, cost_one, cost_two, cost_three, **kwargs):
        n_one, _   = cost_one.shape

        self.one = cost_one
        self.two = cost_two
        self.three = cost_three
        
        super().__init__(n_var=n_one, n_obj=3, xl=0, xu=n_one, vtype=int, **kwargs)

    def _evaluate(self, x, out, *args, **kwargs):
        out['F'] = self.get_route_length(x)

    def get_route_length(self, x):
        n_cities = len(x)
        dist_one = 0
        dist_two = 0
        dist_three = 0
        
        for k in range(n_cities - 1):
            i, j = x[k], x[k + 1]
            dist_one += self.one[i, j]
            dist_two += self.two[i, j]
            dist_three += self.three[i, j]

        last, first = x[-1], x[0]
        
        dist_one += self.one[last, first]
        dist_one = 100000/dist_one
        dist_two += self.two[last, first]
        dist_two = 100000/dist_two
        dist_three += self.three[last, first]
        dist_three = 100000/dist_three
        
        return dist_one, dist_two, dist_three

In [9]:
cost_one = kroA100.values
cost_two = kroB100.values
cost_three = kroC100.values

one = cdist(cost_one,cost_one)
two = cdist(cost_two,cost_two)
three = cdist(cost_three,cost_three)

In [10]:
problem = TravelingSalesman(cost_one=one, cost_two=two, cost_three=three)

In [11]:
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=12)

In [ ]:
algorithm_nsga3 = NSGA3(pop_size=91,
                        sampling=PermutationRandomSampling(),
                        selection=TournamentSelection(func_comp=comp_by_cv_then_random),
                        crossover=OrderCrossover(),
                        mutation=InversionMutation(),
                        ref_dirs=ref_dirs,
                        eliminate_duplicates=False)

In [12]:
algorithm_nsga2 = NSGA2(pop_size=92,
                        sampling=PermutationRandomSampling(),
                        selection=TournamentSelection(func_comp=binary_tournament),
                        crossover=OrderCrossover(),
                        mutation=InversionMutation(),
                        eliminate_duplicates=False)

In [ ]:
res_motsp_nsga3 = minimize(problem, 
                           algorithm_nsga3, 
                           ('n_gen', 5000), 
                           verbose=True, 
                           save_history=False,
                           seed=1)

In [ ]:
with open('pymoo_pop_nsga3_motsp_91_500_3_100.csv', 'w', newline='') as f, open('pymoo_fitness_nsga3_motsp_91_500_3_100.csv','w', newline='') as g:
    writer = csv.writer(f)    
    writerFitness = csv.writer(g)
    writer.writerow(["f1","f2","f3"])
    
    for fitness in res_motsp_nsga3.F:
            writerFitness.writerow(fitness)
            
    for pop in res_motsp_nsga3.X:
            writer.writerow(pop)

In [13]:
res_motsp_nsga2 = minimize(problem, 
                           algorithm_nsga2, 
                           ('n_gen', 5000), 
                           verbose=True, 
                           save_history=False,
                           seed=1)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       92 |     15 |             - |             -
     2 |      184 |     20 |  0.0767215185 |         ideal
     3 |      276 |     28 |  0.0524553418 |         ideal
     4 |      368 |     36 |  0.0762536899 |         ideal
     5 |      460 |     36 |  0.0760831550 |         ideal
     6 |      552 |     39 |  0.0785088853 |         ideal
     7 |      644 |     40 |  0.2980609305 |         nadir
     8 |      736 |     40 |  0.0770205689 |         ideal
     9 |      828 |     49 |  0.0545788724 |         ideal
    10 |      920 |     44 |  0.0302874598 |         ideal
    11 |     1012 |     43 |  0.0068956270 |         nadir
    12 |     1104 |     57 |  0.0460542310 |         ideal
    13 |     1196 |     54 |  0.0352874699 |         ideal
    14 |     1288 |     60 |  0.0522426892 |         ideal
    15 |     1380 |     63 |  0.0352111538 |         ideal
    16 |     1472 |     52 |  0.0225235209 |         ide

   143 |    13156 |     92 |  0.0036368543 |             f
   144 |    13248 |     92 |  0.0971911913 |         nadir
   145 |    13340 |     92 |  0.0148545192 |         nadir
   146 |    13432 |     92 |  0.0055643489 |         ideal
   147 |    13524 |     92 |  0.0036476361 |             f
   148 |    13616 |     92 |  0.0019404703 |             f
   149 |    13708 |     92 |  0.0105695201 |             f
   150 |    13800 |     92 |  0.0025241632 |             f
   151 |    13892 |     92 |  0.0015229664 |             f
   152 |    13984 |     92 |  0.0051238896 |             f
   153 |    14076 |     92 |  0.0102952126 |         ideal
   154 |    14168 |     92 |  0.0032451076 |         ideal
   155 |    14260 |     92 |  0.0057071803 |             f
   156 |    14352 |     92 |  0.0617460620 |         nadir
   157 |    14444 |     92 |  0.0040998603 |             f
   158 |    14536 |     92 |  0.0033365487 |             f
   159 |    14628 |     92 |  0.0026996898 |            

   284 |    26128 |     92 |  0.0035151764 |             f
   285 |    26220 |     92 |  0.0105077996 |         ideal
   286 |    26312 |     92 |  0.0043814295 |             f
   287 |    26404 |     92 |  0.0711293624 |         nadir
   288 |    26496 |     92 |  0.0044982927 |             f
   289 |    26588 |     92 |  0.0051012567 |         nadir
   290 |    26680 |     92 |  0.0609693255 |         nadir
   291 |    26772 |     92 |  0.0580894347 |         nadir
   292 |    26864 |     92 |  0.0193784262 |         nadir
   293 |    26956 |     92 |  0.0039986197 |         ideal
   294 |    27048 |     92 |  0.0049678870 |             f
   295 |    27140 |     92 |  0.0024819982 |             f
   296 |    27232 |     92 |  0.0075538075 |             f
   297 |    27324 |     92 |  0.0035993225 |             f
   298 |    27416 |     92 |  0.0034151613 |             f
   299 |    27508 |     92 |  0.0058960393 |             f
   300 |    27600 |     92 |  0.0051622955 |            

   426 |    39192 |     92 |  0.0073668363 |             f
   427 |    39284 |     92 |  0.0054607470 |             f
   428 |    39376 |     92 |  0.0092329803 |             f
   429 |    39468 |     92 |  0.0071090840 |             f
   430 |    39560 |     92 |  0.0034198371 |             f
   431 |    39652 |     92 |  0.0069735351 |         ideal
   432 |    39744 |     92 |  0.0084157028 |             f
   433 |    39836 |     92 |  0.0093450091 |             f
   434 |    39928 |     92 |  0.0092665302 |             f
   435 |    40020 |     92 |  0.0076596447 |             f
   436 |    40112 |     92 |  0.0327521869 |         nadir
   437 |    40204 |     92 |  0.0053388427 |             f
   438 |    40296 |     92 |  0.0090183357 |             f
   439 |    40388 |     92 |  0.0077524926 |             f
   440 |    40480 |     92 |  0.0062156888 |             f
   441 |    40572 |     92 |  0.0058039327 |             f
   442 |    40664 |     92 |  0.0058388445 |            

   566 |    52072 |     92 |  0.0079518065 |             f
   567 |    52164 |     92 |  0.0553113893 |         nadir
   568 |    52256 |     92 |  0.0036472136 |         nadir
   569 |    52348 |     92 |  0.0036605645 |         nadir
   570 |    52440 |     92 |  0.0050278013 |             f
   571 |    52532 |     92 |  0.0827142277 |         nadir
   572 |    52624 |     92 |  0.0091068026 |             f
   573 |    52716 |     92 |  0.0060049388 |             f
   574 |    52808 |     92 |  0.0087552805 |             f
   575 |    52900 |     92 |  0.0071780642 |             f
   576 |    52992 |     92 |  0.0069951283 |             f
   577 |    53084 |     92 |  0.0075618006 |             f
   578 |    53176 |     92 |  0.0469263505 |         nadir
   579 |    53268 |     92 |  0.0055160884 |             f
   580 |    53360 |     92 |  0.0077036641 |             f
   581 |    53452 |     92 |  0.0232384892 |         nadir
   582 |    53544 |     92 |  0.0152008310 |         nad

...

  4397 |   404524 |     92 |  0.0062849049 |             f
  4398 |   404616 |     92 |  0.0108069279 |             f
  4399 |   404708 |     92 |  0.0068458291 |             f
  4400 |   404800 |     92 |  0.0055440693 |         nadir
  4401 |   404892 |     92 |  0.0083462265 |             f
  4402 |   404984 |     92 |  0.0178918031 |         nadir
  4403 |   405076 |     92 |  0.0384545090 |         nadir
  4404 |   405168 |     92 |  0.0090548831 |             f
  4405 |   405260 |     92 |  0.0173543755 |         nadir
  4406 |   405352 |     92 |  0.0182177516 |         nadir
  4407 |   405444 |     92 |  0.0090750590 |             f
  4408 |   405536 |     92 |  0.0094626435 |             f
  4409 |   405628 |     92 |  0.0107622301 |             f
  4410 |   405720 |     92 |  0.0175248873 |         nadir
  4411 |   405812 |     92 |  0.0072809401 |             f
  4412 |   405904 |     92 |  0.0048865318 |         nadir
  4413 |   405996 |     92 |  0.0092779390 |            

  4538 |   417496 |     92 |  0.0055900755 |         nadir
  4539 |   417588 |     92 |  0.0072875500 |             f
  4540 |   417680 |     92 |  0.0031169804 |             f
  4541 |   417772 |     92 |  0.0059546442 |             f
  4542 |   417864 |     92 |  0.0033708978 |         nadir
  4543 |   417956 |     92 |  0.0099321598 |             f
  4544 |   418048 |     92 |  0.0072006579 |             f
  4545 |   418140 |     92 |  0.0065762349 |             f
  4546 |   418232 |     92 |  0.0077550452 |             f
  4547 |   418324 |     92 |  0.0117909259 |             f
  4548 |   418416 |     92 |  0.0116120323 |             f
  4549 |   418508 |     92 |  0.0071005812 |             f
  4550 |   418600 |     92 |  0.0132448717 |         nadir
  4551 |   418692 |     92 |  0.0500737997 |         nadir
  4552 |   418784 |     92 |  0.0063259699 |             f
  4553 |   418876 |     92 |  0.0085235474 |             f
  4554 |   418968 |     92 |  0.0077146111 |            

  4678 |   430376 |     92 |  0.0084699598 |             f
  4679 |   430468 |     92 |  0.0088553965 |             f
  4680 |   430560 |     92 |  0.0250095637 |         nadir
  4681 |   430652 |     92 |  0.0022726052 |             f
  4682 |   430744 |     92 |  0.0177291227 |         nadir
  4683 |   430836 |     92 |  0.0074652409 |             f
  4684 |   430928 |     92 |  0.0107023483 |             f
  4685 |   431020 |     92 |  0.0092779692 |             f
  4686 |   431112 |     92 |  0.0120740374 |             f
  4687 |   431204 |     92 |  0.0064009249 |             f
  4688 |   431296 |     92 |  0.0052849100 |             f
  4689 |   431388 |     92 |  0.0077156088 |             f
  4690 |   431480 |     92 |  0.0121247108 |             f
  4691 |   431572 |     92 |  0.0072953698 |             f
  4692 |   431664 |     92 |  0.0112215110 |             f
  4693 |   431756 |     92 |  0.0480661722 |         nadir
  4694 |   431848 |     92 |  0.0057062256 |            

  4821 |   443532 |     92 |  0.0099236647 |             f
  4822 |   443624 |     92 |  0.0652246616 |         nadir
  4823 |   443716 |     92 |  0.0095209915 |         nadir
  4824 |   443808 |     92 |  0.0110164162 |             f
  4825 |   443900 |     92 |  0.0069910966 |         nadir
  4826 |   443992 |     92 |  0.0089436539 |             f
  4827 |   444084 |     92 |  0.0104819053 |             f
  4828 |   444176 |     92 |  0.0103804605 |             f
  4829 |   444268 |     92 |  0.0059112726 |             f
  4830 |   444360 |     92 |  0.0082956700 |             f
  4831 |   444452 |     92 |  0.0119313090 |             f
  4832 |   444544 |     92 |  0.0082730338 |             f
  4833 |   444636 |     92 |  0.0064816737 |             f
  4834 |   444728 |     92 |  0.0053107459 |             f
  4835 |   444820 |     92 |  0.0102249528 |             f
  4836 |   444912 |     92 |  0.0309295233 |         nadir
  4837 |   445004 |     92 |  0.0081754707 |            

  4962 |   456504 |     92 |  0.0106590884 |             f
  4963 |   456596 |     92 |  0.0080782168 |             f
  4964 |   456688 |     92 |  0.0063290445 |             f
  4965 |   456780 |     92 |  0.0061270505 |             f
  4966 |   456872 |     92 |  0.0020634065 |             f
  4967 |   456964 |     92 |  0.0120375685 |             f
  4968 |   457056 |     92 |  0.0192524422 |         nadir
  4969 |   457148 |     92 |  0.0094039876 |             f
  4970 |   457240 |     92 |  0.0071334750 |             f
  4971 |   457332 |     92 |  0.0066577550 |             f
  4972 |   457424 |     92 |  0.0270479383 |         nadir
  4973 |   457516 |     92 |  0.0034900588 |             f
  4974 |   457608 |     92 |  0.0052432090 |             f
  4975 |   457700 |     92 |  0.0040742817 |             f
  4976 |   457792 |     92 |  0.0043271774 |             f
  4977 |   457884 |     92 |  0.0149753926 |         nadir
  4978 |   457976 |     92 |  0.0716455763 |         nad

In [18]:
with open('pymoo_pop_nsga2_motsp_92_500_3_100.csv', 'w', newline='') as f, open('pymoo_fitness_nsga2_motsp_92_500_3_100.csv','w', newline='') as g:
    writer = csv.writer(f)    
    writerFitness = csv.writer(g)
    writer.writerow(["f1","f2","f3"])
    
    for fitness in res_motsp_nsga2.F:
            writerFitness.writerow(fitness)
            
    for pop in res_motsp_nsga2.X:
            writer.writerow(pop)